In [1]:
import repo_chat.git2vectors as git2vectors
from repo_chat.chat_utils import RetrievalChain
from IPython.display import Markdown
import pandas as pd
import json

c:\Users\voyno\Desktop\code\repos\repo-chat\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Create vectorstore, this will take a while
repo = "https://github.com/smol-ai/developer"
# git2vectors.create_vectorstore(repo)

# load vectorstore, this is fas
vectorstore = git2vectors.get_vectorstore()

In [3]:
# Create an instance of the class
chain = RetrievalChain(vectorstore, repo)

# Let's say we have a query
query = "Explain this to me like I\'m 5."

# generic retrieval query
response = chain.chat(query)
Markdown(response['text'])

['# smol developer\r\n\r\n***Human-centric & Coherent Whole Program Synthesis*** aka your own personal junior developer\r\n\r\n> [Build the thing that builds the thing!](https://twitter.com/swyx/status/1657578738345979905) a `smol dev` for every dev in every situation\r\n\r\nthis is a prototype of a "junior developer" agent (aka `smol dev`) that scaffolds an entire codebase out for you once you give it a product spec, but does not end the world or overpromise AGI. instead of making and maintaining specific, rigid, one-shot starters, like `create-react-app`, or `create-nextjs-app`, this is basically [`create-anything-app`](https://news.ycombinator.com/item?id=35942352) where you develop your scaffolding prompt in a tight loop with your smol dev.\r\n\r\nAI that is helpful, harmless, and honest is complemented by a codebase that is simple, safe, and smol - <200 lines of Python and Prompts, so this is easy to understand and customize.\r\n\r\n<p align="center">\r\n  <img height=200 src="htt

This github repository contains a prototype for a "junior developer" agent that scaffolds an entire codebase for you based on a product spec. It is a human-centric and coherent whole program synthesis tool that uses AI to assist in the development process. The codebase is simple, safe, and small, making it easy to understand and customize. The demo example shows how the tool works by allowing the user to write a basic prompt for the app they want to build, generating code, and then manually running the code and identifying errors. The AI is only used as long as it is adding value, and the user can take over the codebase from the "junior developer" at any time. The tool is not no code or low code, but rather a third thing that still requires technical knowledge. The repository also includes a video demo and an architecture diagram. The tool requires Modal.com, GPT-4 API, and anthropic claude 100k context API, which are currently in private beta. The example project is a Chrome extension that summarizes web pages using the Anthropic Claude API.